In [1]:
import os
import pandas as pd
from modules.data_preprocessing import DataPreprocessing
from modules.feature_engineering import FeatureEngineeringV1


In [2]:
data_preprocessing = DataPreprocessing()
fe = FeatureEngineeringV1()

In [3]:
directory_prefix = '../data/excel_raw/'
lst_of_files = [os.path.join(directory_prefix, file) for file in os.listdir(directory_prefix)]
lst_of_files

['../data/excel_raw/20230713T153526004.att.672322581406152.xlsx',
 '../data/excel_raw/20230713T153529650.att.1333037354235214.xlsx']

In [4]:
csv_path='../data/csv/raw.csv'
data_preprocessing.handle_init_files(lst_of_files, csv_path)

The file '../data/csv/raw.csv' already exists. Skipping file creation.


In [5]:
raw = pd.read_csv('../data/csv/raw.csv')
raw

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Route,Ac_id,Ac_code,Ac_type,Ac_kts,Time_step,Phase,Ac_Lat,Ac_Lon,Ac_feet
0,3.2,130.9,1.1,130.3,2.4,202.5,0.8,142.5,1.6,141.7,...,6,R6-VJ150-21,VJ150,1,170.0,3,1,10.8134,106.6427,625.0
1,3.2,130.9,1.1,130.4,2.4,202.9,0.8,142.6,1.6,141.5,...,6,R6-VJ150-21,VJ150,1,179.0,23,1,10.8075,106.6271,1425.0
2,3.2,130.9,1.1,130.4,2.4,202.9,0.8,142.6,1.6,141.5,...,6,R6-VJ150-21,VJ150,1,205.0,43,1,10.8029,106.6096,1800.0
3,3.2,130.9,1.1,130.5,2.4,203.3,0.7,142.8,1.6,141.4,...,6,R6-VJ150-21,VJ150,1,231.0,59,1,10.8063,106.5941,2025.0
4,3.2,130.9,1.1,130.5,2.4,203.3,0.7,142.8,1.6,141.4,...,6,R6-VJ150-21,VJ150,1,255.0,75,1,10.8204,106.5810,2475.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61383,1.9,137.2,1.8,42.6,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,157.0,5344,1,21.2532,105.6876,1775.0
61384,1.9,137.2,1.8,42.6,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,152.0,5362,1,21.2493,105.7010,1525.0
61385,1.9,137.2,1.8,42.2,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,146.0,5378,1,21.2461,105.7121,1325.0
61386,1.9,137.2,1.8,42.2,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,131.0,5412,1,21.2401,105.7331,950.0


In [6]:
processed_df = fe.process_data('../data/csv/raw.csv')
processed_df

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Route,Ac_id,Ac_code,Ac_type,Ac_kts,Time_step,Phase,Ac_Lat,Ac_Lon,Ac_feet
0,0.630435,0.363611,0.200000,0.361868,0.229167,0.561418,0.014493,0.993860,0.150000,0.884247,...,6,R6-VJ150-21,VJ150,1,170.0,3,1,-0.931666,-0.090884,0.012828
1,0.630435,0.363611,0.200000,0.362146,0.229167,0.562535,0.014493,0.994628,0.150000,0.882877,...,6,R6-VJ150-21,VJ150,1,179.0,23,1,-0.932750,-0.101588,0.032376
2,0.630435,0.363611,0.200000,0.362146,0.229167,0.562535,0.014493,0.994628,0.150000,0.882877,...,6,R6-VJ150-21,VJ150,1,205.0,43,1,-0.933595,-0.113597,0.041539
3,0.630435,0.363611,0.200000,0.362424,0.229167,0.563652,0.000000,0.996163,0.150000,0.882192,...,6,R6-VJ150-21,VJ150,1,231.0,59,1,-0.932970,-0.124232,0.047037
4,0.630435,0.363611,0.200000,0.362424,0.229167,0.563652,0.000000,0.996163,0.150000,0.882192,...,6,R6-VJ150-21,VJ150,1,255.0,75,1,-0.930379,-0.133221,0.058033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61383,0.347826,0.381111,0.327273,0.118121,0.531250,0.101619,0.391304,0.629317,0.533333,0.580822,...,6,R6-VN258-04,VN258,2,157.0,5344,1,0.986587,-0.746252,0.040929
61384,0.347826,0.381111,0.327273,0.118121,0.531250,0.101619,0.391304,0.629317,0.533333,0.580822,...,6,R6-VN258-04,VN258,2,152.0,5362,1,0.985870,-0.737057,0.034820
61385,0.347826,0.381111,0.327273,0.117009,0.531250,0.101619,0.391304,0.629317,0.533333,0.580822,...,6,R6-VN258-04,VN258,2,146.0,5378,1,0.985282,-0.729440,0.029933
61386,0.347826,0.381111,0.327273,0.117009,0.531250,0.101619,0.391304,0.629317,0.533333,0.580822,...,6,R6-VN258-04,VN258,2,131.0,5412,1,0.984180,-0.715031,0.020770


In [7]:
decoded_df = fe.decode_features(processed_df.copy())
decoded_df

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Route,Ac_id,Ac_code,Ac_type,Ac_kts,Time_step,Phase,Ac_Lat,Ac_Lon,Ac_feet
0,3.2,130.9,1.1,130.3,2.4,202.5,0.8,142.5,1.6,141.7,...,6,R6-VJ150-21,VJ150,1,170.0,3,1,10.8134,106.6427,625.0
1,3.2,130.9,1.1,130.4,2.4,202.9,0.8,142.6,1.6,141.5,...,6,R6-VJ150-21,VJ150,1,179.0,23,1,10.8075,106.6271,1425.0
2,3.2,130.9,1.1,130.4,2.4,202.9,0.8,142.6,1.6,141.5,...,6,R6-VJ150-21,VJ150,1,205.0,43,1,10.8029,106.6096,1800.0
3,3.2,130.9,1.1,130.5,2.4,203.3,0.7,142.8,1.6,141.4,...,6,R6-VJ150-21,VJ150,1,231.0,59,1,10.8063,106.5941,2025.0
4,3.2,130.9,1.1,130.5,2.4,203.3,0.7,142.8,1.6,141.4,...,6,R6-VJ150-21,VJ150,1,255.0,75,1,10.8204,106.5810,2475.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61383,1.9,137.2,1.8,42.6,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,157.0,5344,1,21.2532,105.6876,1775.0
61384,1.9,137.2,1.8,42.6,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,152.0,5362,1,21.2493,105.7010,1525.0
61385,1.9,137.2,1.8,42.2,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,146.0,5378,1,21.2461,105.7121,1325.0
61386,1.9,137.2,1.8,42.2,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,131.0,5412,1,21.2401,105.7331,950.0


In [8]:
processed_df.to_csv('../data/csv/processed.csv', index=False)